In [1]:
import EstimatorPreprocessor as ep
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
data = ep.load_cleaned_submissions()
y = ep.encode_labels(data)
X = ep.vectorize_text(data)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [4]:
del X, y, data

In [59]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV

pipeline = Pipeline([
        ("mnb", MultinomialNB())
    ])

param_grid = {"mnb__alpha": np.arange(0, 1.1, 0.1), "mnb__fit_prior": [True, False]}
CV = GridSearchCV(pipeline, param_grid, cv = 5)
# pipeline.get_params().keys() See all available parameters
CV.fit(X_train, y_train)

c:\DEV\Master Thesis\App\.thesis\lib\site-packages\sklearn\naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '
c:\DEV\Master Thesis\App\.thesis\lib\site-packages\sklearn\naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '
c:\DEV\Master Thesis\App\.thesis\lib\site-packages\sklearn\naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '
c:\DEV\Master Thesis\App\.thesis\lib\site-packages\sklearn\naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '
c:\DEV\Master Thesis\App\.thesis\lib\site-packages\sklearn\naive_bayes.py:508: UserW

GridSearchCV(cv=5, estimator=Pipeline(steps=[('mnb', MultinomialNB())]),
             param_grid={'mnb__alpha': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
                         'mnb__fit_prior': [True, False]})

In [60]:
print("Accuracy of {} can be achieved with the following parameters: {}".format(CV.score(X_test, y_test), CV.best_params_))

Accuracy of 0.7991032888690858 can be achieved with the following parameters: {'mnb__alpha': 1.0, 'mnb__fit_prior': False}


In [61]:
y_pred = CV.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.57      0.79      0.66      4960
           1       0.91      0.81      0.86     24104
           2       0.68      0.75      0.71      6845

    accuracy                           0.80     35909
   macro avg       0.72      0.79      0.75     35909
weighted avg       0.82      0.80      0.81     35909



In [62]:
# Todo
# Access all scores of the CV to create cool visualization

In [63]:
CV.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_mnb__alpha', 'param_mnb__fit_prior', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [152]:
textfile = open("./eval_model/naive_bayes_final.txt", "w")
for i, param in enumerate(CV.cv_results_["params"]):
    textfile.write(str(param) + ";" + str(CV.cv_results_["mean_test_score"][i]) + "\n")
textfile.close()

In [153]:
import pandas as pd
df = pd.DataFrame({
    "Params": [item for item in CV.cv_results_["params"]],
    "Accuracy": [item for item in CV.cv_results_["mean_test_score"]]
})

In [154]:
df["Params"] = df["Params"].astype(str)

In [155]:
df["Params"] = df["Params"].str.replace("mnb__", "")
df["Params"] = df["Params"].str.replace("'", "")

In [157]:
df.to_csv("./eval_model/naive_bayes_final.csv", sep='\t')